In [1]:
import numpy as np
from pydub import AudioSegment
import random
import sys
import io
import os
import glob
import IPython
from td_utils import *
import time
import tensorflow as tf
%matplotlib inline

/usr/local/lib/python3.5/dist-packages/pydub/utils.py:165: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
Tx = 5511 # The number of time steps input to the model from the spectrogram
n_freq = 101 # Number of frequencies input to the model at each time step of the spectrogram
Ty = 1375 # The number of time steps in the output of our model

In [3]:
# Load preprocessed training examples
X = np.load("./XY_Train/XTrain_Final3.npy")
Y = np.load("./XY_Train/YTrain_Final3.npy")

In [4]:
from keras.callbacks import ModelCheckpoint
from keras.models import Model, load_model, Sequential
from keras.layers import Dense, Activation, Dropout, Input, Masking, TimeDistributed, LSTM, Conv1D
from keras.layers import GRU, Bidirectional, BatchNormalization, Reshape
from keras.optimizers import Adam

Using TensorFlow backend.


In [5]:
import os
os.environ['TF_ENABLE_AUTO_MIXED_PRECISION'] = '0'

In [6]:

def model(input_shape):
    """
    Function creating the model's graph in Keras.
    
    Argument:
    input_shape -- shape of the model's input data (using Keras conventions)

    Returns:
    model -- Keras model instance
    """
    
    X_input = Input(shape = input_shape)
    
       
    # Step 1: CONV layer (≈4 lines)
    X = Conv1D(196, kernel_size=15, strides=4)(X_input)                                 # CONV1D
    X = BatchNormalization()(X)                                 # Batch normalization
    X = Activation('relu')(X)                                 # ReLu activation
    X = Dropout(0.8)(X)                                 # dropout (use 0.8)

    # Step 2: First GRU Layer (≈4 lines)
    X = GRU(units = 128, return_sequences = True)(X) # GRU (use 128 units and return the sequences)
    X = Dropout(0.8)(X)                                 # dropout (use 0.8)
    X = BatchNormalization()(X)                                 # Batch normalization
    
    # Step 3: Second GRU Layer (≈4 lines)
    X = GRU(units = 128, return_sequences = True)(X)   # GRU (use 128 units and return the sequences)
    X = Dropout(0.8)(X)                                 # dropout (use 0.8)
    X = BatchNormalization()(X)                                  # Batch normalization
    X = Dropout(0.8)(X)                                  # dropout (use 0.8)
    
    # Step 4: Time-distributed dense layer (≈1 line)
    X = TimeDistributed(Dense(1, activation = "sigmoid"))(X) # time distributed  (sigmoid)


    model = Model(inputs = X_input, outputs = X)
    
    return model  

In [7]:
model = model(input_shape = (Tx, n_freq))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [8]:
model = load_model('./tr_model.h5')

Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.5/dist-packages/keras/engine/saving.py:327: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [9]:
opt = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, decay=0.01)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=["accuracy"])

In [10]:
start=time.time()
model.fit(X, Y, batch_size =64, epochs=10)
print(time.time()-start)

Epoch 1/10
70/70 [==============================] - 12s 171ms/step - loss: 0.2387 - acc: 0.9379
Epoch 2/10
70/70 [==============================] - 8s 115ms/step - loss: 0.2113 - acc: 0.9440
Epoch 3/10
70/70 [==============================] - 10s 137ms/step - loss: 0.2013 - acc: 0.9461
Epoch 4/10
70/70 [==============================] - 9s 122ms/step - loss: 0.2049 - acc: 0.9458
Epoch 5/10
70/70 [==============================] - 8s 118ms/step - loss: 0.2099 - acc: 0.9478
Epoch 6/10
70/70 [==============================] - 8s 116ms/step - loss: 0.2056 - acc: 0.9456
Epoch 7/10
70/70 [==============================] - 9s 130ms/step - loss: 0.1930 - acc: 0.9481
Epoch 8/10
70/70 [==============================] - 8s 120ms/step - loss: 0.1947 - acc: 0.9516
Epoch 9/10
70/70 [==============================] - 8s 118ms/step - loss: 0.1801 - acc: 0.9525
Epoch 10/10
70/70 [==============================] - 8s 116ms/step - loss: 0.1820 - acc: 0.9510
90.58503746986389
